In [1]:
import requests
import pandas as pd
import os
import datetime as dt
import matplotlib.pyplot as plt
import urllib3
from bs4 import BeautifulSoup
import requests

## 에어코리아 오픈 API 크롤링

공공 데이터 포털 - 한국환경공단 대기오염정보 오픈 API 이용하여 크롤링 시행

(https://www.data.go.kr/dataset/15000581/openapi.do)

API Key는 해당사이트의 활용신청 후 즉시 발급

하나의 키당 하루 최대 트래픽 300건

In [2]:
key = 'yr0h%2Fc6roehUB54pxZQh4TFzw7tHIsXlP6AhbGwvLkr3b1M7HPDBBKVpsu7mCK5LoVGmwBM2sOqlSgD2wTd%2Ffg%3D%3D'

### 측정소 정보

: 에어코리아 - 에어코리아란 - 측정소 정보에 있는 엑셀 파일 활용하여 지역 리스트 확인

(https://www.airkorea.or.kr/web/stationInfo?pMENU_NO=93)

In [3]:
site_df = pd.read_excel('D:/data/airkorea/station_list.xls')
site_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,측정소 리스트,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,지역명,측정소명,측정소 주소,운영기관,설치년도
3,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978
4,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997


header가 깨져있는 형태, column 형식을 맞추고, 추출할 지역을 지역명 단위로 시행

(셀레니움 이용 시 측정소 주소를 통해 크롤링하여 과거 데이터를 추출)

In [4]:
site_df.columns = site_df.iloc[2,:].tolist()
site_df.drop([0,1,2],inplace=True)
site_df.drop(['측정소 주소','운영기관','설치년도'],axis=1,inplace=True)
site_df.head()

,지역명,측정소명
3,서울,강남구
4,서울,강동구
5,서울,강북구
6,서울,강서구
7,서울,관악구


지역에서 사용하는 측정소명만 리스트로 추출 (EX) 경기만 추출

혹은 측정소명을 바탕으로 검색하여 추출

In [5]:
site_df['지역명'].unique().tolist()

['서울',
 '경기',
 '인천',
 '강원',
 '충남',
 '대전',
 '충북',
 '세종',
 '부산',
 '울산',
 '대구',
 '경북',
 '경남',
 '전남',
 '광주',
 '전북',
 '제주']

In [6]:
site_df =site_df[site_df['지역명']=='경기']
site_df.head()

,지역명,측정소명
28,경기,가평
29,경기,경안동
30,경기,고색동
31,경기,고읍
32,경기,고잔동


In [7]:
site_list = site_df.측정소명.values.tolist()
site_list 

['가평',
 '경안동',
 '고색동',
 '고읍',
 '고잔동',
 '고천동',
 '고촌읍',
 '과천동',
 '광교동',
 '교문동',
 '금곡동',
 '금촌동',
 '기흥',
 '김량장동',
 '남양읍',
 '내동',
 '단대동',
 '당동',
 '대부동',
 '대야동',
 '동구동',
 '동탄',
 '모현읍',
 '목감동',
 '백석읍',
 '백암면',
 '별내동',
 '별양동',
 '보산동',
 '복정동',
 '본오동',
 '봉산동',
 '부곡3동',
 '부곡동1',
 '부림동',
 '비전동',
 '사우동',
 '산본동',
 '상대원동',
 '선단동',
 '소사본동',
 '소하동',
 '송북동',
 '수내동',
 '수지',
 '시화산단',
 '식사동',
 '신원동',
 '신장동',
 '신풍동',
 '안양2동',
 '안양6동',
 '안중',
 '양평',
 '여주',
 '연천',
 '영통동',
 '오남읍',
 '오산동',
 '오정동',
 '우정읍',
 '운정',
 '운중동',
 '원곡동',
 '원시동',
 '의정부1동',
 '의정부동',
 '이동읍',
 '인계동',
 '일동면',
 '장호원읍',
 '정왕동',
 '정자동',
 '주엽동',
 '중2동',
 '창전동',
 '천천동',
 '철산동',
 '청계동',
 '통진읍',
 '평택항',
 '한강신도시',
 '행신동',
 '향남읍',
 '호계동',
 '호매실동',
 '호수동',
 '화도읍']

In [8]:
site_list= ['시화산단']
site_list

['시화산단']

## airkorea_openapi_guide-v1_7_2 참고하여 크롤링

2.2. 대기오염정보 조회 서비스 (p.12)

(3) 시도별 실시간 측정정보 조회 오퍼레이션 명세 (p.18)

[각 측정소별 대기오염정보를 조회하기 위한 서비스로 기간별, 시도별 대기오염 정보와 통합대기환경지수 나쁨 이상 측정소 내역, 대기질(미세먼지/오존) 예보 통보 내역 등을 조회할 수 있다.]

버전 1.1 이상을 호출할 경우 PM10, PM2.5 24시간 예측이동 평균데이터가 포함.

- 버전 1.1을 호출할 경우 : PM10, PM2.5 24시간 예측이동 평균데이터가 포함된 결과 표출.
- 버전 1.2을 호출할 경우 : 측정망 정보 데이터가 포함된 결과 표출.
- 버전 1.3을 호출할 경우 : PM10, PM2.5 1시간 등급 자료가 포함된 결과 표출

버전 1.3 이용하여 데이터 및 1시간 단위 대기 API를 포함하여 추출 (3개월)

## 추출할 데이터 셋 List

airkorea_openapi_guide-v1_7_2의 p.14참고하여 List 작성

In [9]:
pmInfor = {}  #최종 df에 담기 위한 dictionary 생성
dataTimelist = [] # 측정일 
mangNamelist= [] # 측정망 정보 
so2Valuelist = [] # 아황산가스 
coValuelist = [] # 일산화탄소 
o3Valuelist = [] # 오존 
no2Valuelist = [] # 이산화질소 
pm10Valuelist = [] # PM10  
pm25Valuelist = [] # PM2.5 
pm10Value24list = [] #PM10 24시간 예측이동농도
pm25Value24list = [] #PM2.5 24시간 예측이동농도


### Data Crawling

In [10]:
# crawling_list = ['datatime','mangname','so2value','covalue','o3value','no2value','pm10value','pm25value',
#                 'pm10value24','pm25value24']
# list_data = ['dataTimelist','mangNamelist','so2Valuelist','coValuelist','o3Valuelist','no2Valuelist',
#             'pm10Valuelist','pm25Valuelist','pm10Value24list','pm25Value24list']


for site, value in zip(site_list,range(len(site_list))) :
    for k in range(1,10) :
        url=f'http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?stationName={site}&dataTerm=3month&pageNo={k}&numOfRows=500&ServiceKey={key}&ver=1.3'
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        dataTime = soup.find_all('datatime')
        mangName = soup.find_all('mangname')
        so2Value = soup.find_all('so2value')
        coValue = soup.find_all('covalue')
        o3Value = soup.find_all('o3value')
        no2Value = soup.find_all('no2value')
        pm10Value = soup.find_all('pm10value')
        pm25Value = soup.find_all('pm25value')
        pm10Value24 = soup.find_all('pm10value24')
        pm25Value24 = soup.find_all('pm25value24')
        for code in dataTime:
            dataTimelist.append(code.text)
        for code in mangName:
            mangNamelist.append(site_list[value])
        for code in so2Value:
            so2Valuelist.append(code.text)
        for code in coValue:
            coValuelist.append(code.text)
        for code in o3Value:
            o3Valuelist.append(code.text)
        for code in no2Value:
            no2Valuelist.append(code.text)
        for code in pm10Value:
            pm10Valuelist.append(code.text)
        for code in pm25Value:
            pm25Valuelist.append(code.text)
        for code in pm10Value24:
            pm10Value24list.append(code.text)
        for code in pm25Value24:
            pm25Value24list.append(code.text)


In [11]:
pmInfor['datatime'] = dataTimelist
pmInfor['mangname'] = mangNamelist
pmInfor['so2value'] = so2Valuelist
pmInfor['covalue'] = coValuelist
pmInfor['o3value'] = o3Valuelist
pmInfor['no2value'] = no2Valuelist
pmInfor['pm10value'] = pm10Valuelist
pmInfor['pm25value'] = pm25Valuelist
pmInfor['pm10value24'] = pm10Value24list
pmInfor['pm25value24'] = pm25Value24list
pm= pd.DataFrame(pmInfor)
pm.head(30)

,datatime,mangname,so2value,covalue,o3value,no2value,pm10value,pm25value,pm10value24,pm25value24
0,2020-01-30 15:00,시화산단,0.002,0.4,0.032,0.022,39,12,29,8
1,2020-01-30 14:00,시화산단,0.003,0.4,0.025,0.031,40,11,28,8
2,2020-01-30 13:00,시화산단,0.003,0.4,0.021,0.033,29,9,26,8
3,2020-01-30 12:00,시화산단,0.003,0.4,0.022,0.029,20,7,26,8
4,2020-01-30 11:00,시화산단,0.003,0.4,0.016,0.035,35,10,26,7
5,2020-01-30 10:00,시화산단,0.004,0.6,0.007,0.046,31,12,24,6
6,2020-01-30 09:00,시화산단,0.004,0.6,0.005,0.048,28,8,21,5
7,2020-01-30 08:00,시화산단,0.003,0.5,0.004,0.045,20,4,20,4
8,2020-01-30 07:00,시화산단,0.002,0.3,0.011,0.034,18,2,19,4
9,2020-01-30 06:00,시화산단,0.002,0.3,0.022,0.021,14,2,19,5


In [16]:
predict = pm[['datatime','pm25value','pm25value24']]
predict = predict.sort_values(['datatime'])
predict.head()

,datatime,pm25value,pm25value24
2204,2019-10-30 18:00,7,9
2203,2019-10-30 19:00,13,10
2202,2019-10-30 20:00,15,11
2201,2019-10-30 21:00,20,13
2200,2019-10-30 22:00,25,16


In [14]:
predict = pm[['datatime','pm25value','pm25value24']]
predict.to_csv('D:/data/pm_api_2020.csv',index=False)

## airkorea_openapi_guide-v1_7_2

2.2. 대기오염정보 조회 서비스 (p.12)

(4) 대기질 예보통보 조회 오퍼레이션 명세 (p.21)

  - 미세먼지 예보는 오늘예보 / 내일예보 / 모레예보가 제공되며, 시시각각으로 변하는 대기질 상황을 전달하기 위해 매일 4회(오전5시, 오전 11시, 오후5시(17시), 오후11시(23시))에 18개 권역으로 발표되고 있습니다. (api_guide 내용)
  
  [' 보통', ' 좋음', ' 나쁨', ' 매우나쁨'] 4단계로 구성
  (좋음(0-50), 보통(51-100), 나쁨(101-250), 매우나쁨(250-))

In [13]:
dustInfor = {}  #최종 df에 담기 위한 dictionary 생성
dataTimelist = [] # 예보 일자
informCodelist= [] #  통보 코드
informGradelist = [] # 예보등급 
informDatalist = [] # 예측 통보 시간

In [14]:
start_date = '2019-09-17'
end_date = '2019-12-17'
for date in pd.date_range(start_date,end_date).strftime('%Y-%m-%d') :
    url=f'http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMinuDustFrcstDspth?searchDate={date}&ServiceKey={key}'
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    dataTime = soup.find_all('datatime')
    informCode = soup.find_all('informcode')
    informGrade = soup.find_all('informgrade')
    informData = soup.find_all('informdata')
    for code in dataTime:
        dataTimelist.append(code.text)
    for code in informCode:
        informCodelist.append(site_list[value])
    for code in informGrade:
        informGradelist.append(code.text)
    for code in informData:
        informDatalist.append(code.text)

In [15]:
dustInfor['datatime'] = dataTimelist
dustInfor['informcode'] = informCodelist
dustInfor['informgrade'] = informGradelist
dustInfor['informdata'] = informDatalist
pm= pd.DataFrame(dustInfor)
pm.head()

,datatime,informcode,informgrade,informdata
0,2019-11-18 23시 발표,시화산단,"서울 : 보통,제주 : 보통,전남 : 보통,전북 : 보통,광주 : 보통,경남 : 보...",2019-11-18
1,2019-11-18 23시 발표,시화산단,"서울 : 좋음,제주 : 보통,전남 : 보통,전북 : 보통,광주 : 보통,경남 : 보...",2019-11-19
2,2019-11-18 23시 발표,시화산단,"서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2019-11-20
3,2019-11-18 23시 발표,시화산단,"서울 : 좋음,제주 : 보통,전남 : 좋음,전북 : 보통,광주 : 좋음,경남 : 좋...",2019-11-18
4,2019-11-18 23시 발표,시화산단,"서울 : 좋음,제주 : 보통,전남 : 보통,전북 : 좋음,광주 : 좋음,경남 : 좋...",2019-11-19
5,2019-11-18 23시 발표,시화산단,"서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2019-11-20
6,2019-11-18 17시 발표,시화산단,"서울 : 나쁨,제주 : 나쁨,전남 : 나쁨,전북 : 나쁨,광주 : 나쁨,경남 : 나...",2019-11-18
7,2019-11-18 17시 발표,시화산단,"서울 : 좋음,제주 : 보통,전남 : 보통,전북 : 보통,광주 : 보통,경남 : 보...",2019-11-19
8,2019-11-18 17시 발표,시화산단,"서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...",2019-11-20
9,2019-11-18 17시 발표,시화산단,"서울 : 보통,제주 : 보통,전남 : 보통,전북 : 보통,광주 : 보통,경남 : 보...",2019-11-18


In [16]:
test = pm['informgrade'].str.split(',',expand = True)

In [17]:
test_list = test.columns.tolist()
df = pd.DataFrame()
for i in test_list:
    data = test[i].str.split(':',expand = True)
    site = data[0].iloc[1].strip()
    data.rename(columns={1:site},inplace=True)
    data = data[site]
    df = pd.concat([df,data],axis=1)

In [18]:
df.head()

,서울,제주,전남,전북,광주,경남,경북,울산,대구,부산,충남,충북,세종,대전,영동,영서,경기남부,경기북부,인천
0,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,나쁨,보통,보통,보통,보통,나쁨,나쁨,보통
1,좋음,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,보통,좋음,좋음,좋음,좋음,좋음
2,보통,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,보통,보통,좋음,좋음,좋음,좋음,보통,보통,보통
3,좋음,보통,좋음,보통,좋음,좋음,보통,좋음,좋음,좋음,좋음,보통,좋음,보통,좋음,좋음,좋음,좋음,좋음
4,좋음,보통,보통,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음


In [20]:
pm_forecast = pd.concat([pm,df],axis=1)
pm_forecast.drop('informgrade',axis=1,inplace=True)
pm_forecast.head()

,datatime,informcode,informdata,서울,제주,전남,전북,광주,경남,경북,...,부산,충남,충북,세종,대전,영동,영서,경기남부,경기북부,인천
0,2019-11-18 23시 발표,시화산단,2019-11-18,보통,보통,보통,보통,보통,보통,보통,...,보통,보통,나쁨,보통,보통,보통,보통,나쁨,나쁨,보통
1,2019-11-18 23시 발표,시화산단,2019-11-19,좋음,보통,보통,보통,보통,보통,보통,...,보통,보통,보통,보통,보통,좋음,좋음,좋음,좋음,좋음
2,2019-11-18 23시 발표,시화산단,2019-11-20,보통,좋음,좋음,좋음,좋음,좋음,좋음,...,좋음,보통,보통,좋음,좋음,좋음,좋음,보통,보통,보통
3,2019-11-18 23시 발표,시화산단,2019-11-18,좋음,보통,좋음,보통,좋음,좋음,보통,...,좋음,좋음,보통,좋음,보통,좋음,좋음,좋음,좋음,좋음
4,2019-11-18 23시 발표,시화산단,2019-11-19,좋음,보통,보통,좋음,좋음,좋음,좋음,...,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음,좋음


In [21]:
pm_forecast['서울'].unique().tolist()

[' 보통', ' 좋음', ' 나쁨', ' 매우나쁨']